In [ ]:
import pandas as pd 
look = pd.read_csv(r"Strategy Input\BasicNoDeviations-4to8Decks-HitSoft17.csv", delimiter='\t')
look.iloc[0,1:] = look.columns[1:]
look.to_csv("BasicNoDeviations-4to8Decks-HitSoft17.csv", header=False, index=False)

In [ ]:
%%cython --cplus

from libcpp.unordered_map cimport unordered_map # we use the map from the C++ STL - quicker than Python dict? 

ctypedef unordered_map[char, Py_ssize_t] parse_char_t # create a mapping type for our char bytes
ctypedef unordered_map[int, Py_ssize_t] get_idx_t # get indexes based on hard values

# Map dealer upcard to vertical (second dimension - C order) index
cdef parse_char_t dealer_idx
dealer_idx.insert({'2', 0})
dealer_idx.insert({'3', 1})
dealer_idx.insert({'4', 2})
dealer_idx.insert({'5', 3})
dealer_idx.insert({'6', 4})
dealer_idx.insert({'7', 5})
dealer_idx.insert({'8', 6})
dealer_idx.insert({'9', 7})
dealer_idx.insert({'T', 8})
dealer_idx.insert({'J', 8})
dealer_idx.insert({'Q', 8})
dealer_idx.insert({'K', 8})
dealer_idx.insert({'A', 9})

In [ ]:
%%cython --cplus --annotate
# do not know if this global directive is actually needed - maybe later when we break into files
# # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
# cython: c_string_type=str, c_string_encoding=ascii

import numpy as np
cimport cython
from libcpp.vector cimport vector

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# 'Blank card' to fill initial values hand vectors
cdef card_dtype_struct BLANK_CARD = [<char>0, <char>0]

# Python dicts actually implemented quite fast
# If this is too slow, can consider using unordered_map from C++ STL. 
# https://stackoverflow.com/questions/29268914/python-dictionaries-vs-c-stdunordered-map-cython-vs-cythonized-python-dict
# https://stackoverflow.com/questions/32266444/using-a-dictionary-in-cython-especially-inside-nogil
cdef dict dealer_idx = {
    b'2': 0,
    b'3': 1,
    b'4': 2,
    b'5': 3,
    b'6': 4,
    b'7': 5,
    b'8': 6,
    b'9': 7,
    b'T': 8,
    b'J': 8,
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}


cdef int ASCII_A = 65 
cdef unsigned int MEMBLOCKS = 21 # LARGEST POSSIBLE CARD SIZE OF HAND (21 SOFT ACES)

# Use vector from cpp STL to represent player and dealer hands
# Do we save any overhead declaring outside the function?
cdef vector[card_dtype_struct] phand = vector[card_dtype_struct](MEMBLOCKS)
cdef vector[card_dtype_struct] dhand = vector[card_dtype_struct](MEMBLOCKS)

# function decorators to turn off various things, in the interest of speed
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(
    card_dtype_struct[:] shoe not None, 
    Py_ssize_t numcardsdeal, 
    char[:,:] hrd not None, 
    char[:,:] sft not None, 
    char[:,:] splt not None, 
    double[:] cnt not None):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 

    Inputs: (REFER TO TEMPLATE FILES IN STRATEGY INPUT)
        - 'shoe': numpy array (buffer) of char[2] byte strings
        - 'numcardsdeal': int, simulates the depth of the cut card
        - 'hrd': numpy array (buffer) of char bytes, in the input format dictating BJ action on player hard hand
        - 'sft': numpy array (buffer) of char bytes, in the input format dictating BJ action on player soft hand
        - 'splt': numpy array (buffer) of char bytes, in the input format dictating BJ action on player paired hand
        - 'cnt': numpy array (buffer) of double (Python float6), in the input format, with count values
    """

    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef double count = 0 # card count

    # track how many 'real' cards are being stored in the vector
    cdef unsigned int ph 
    cdef unsigned int dh 

    while dealt < numcardsdeal:
    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
        
        # init both our hands to blanks
        phand = vector[card_dtype_struct](MEMBLOCKS, BLANK_CARD)
        dhand = vector[card_dtype_struct](MEMBLOCKS, BLANK_CARD)
        ph_size = 0
        dh_size = 0

        # deal out
        phand[ph_size] = shoe[dealt]
        ph_size += 1
        dealt += 1 
        dhand[dh_size]
        dh_size += 1

        break
    
    print(phand[0])


    print(bool(shoe[0].face == b'J'))

    # type cast into python bytes, so works as dict key
    # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
    print(dealer_idx.get(<bytes>(shoe[0].face)))




In [ ]:
%%cython --cplus
# do not know if this global directive is actually needed - maybe later when we break into files
# # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
# cython: c_string_type=str, c_string_encoding=ascii

import numpy as np
cimport cython

cdef int ASCII_A = 65 
cdef unsigned int MEMBLOCKS = 21 # LARGEST POSSIBLE CARD SIZE OF HAND (21 SOFT ACES)

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# Python dicts actually implemented quite fast
# If this is too slow, can consider using unordered_map from C++ STL. 
# https://stackoverflow.com/questions/29268914/python-dictionaries-vs-c-stdunordered-map-cython-vs-cythonized-python-dict
# https://stackoverflow.com/questions/32266444/using-a-dictionary-in-cython-especially-inside-nogil
cdef dict dealer_idx = {
    b'2': 0,
    b'3': 1,
    b'4': 2,
    b'5': 3,
    b'6': 4,
    b'7': 5,
    b'8': 6,
    b'9': 7,
    b'T': 8,
    b'J': 8,
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}

ctypedef unsigned int[MEMBLOCKS] hand_t

# function decorators to turn off various things, in the interest of speed
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(
    card_dtype_struct[:] shoe not None, 
    Py_ssize_t cut_idx, 
    char[:,:] hrd not None, 
    char[:,:] sft not None, 
    char[:,:] splt not None, 
    double[:] cnt not None):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 

    Inputs: (REFER TO TEMPLATE FILES IN STRATEGY INPUT)
        - 'shoe': numpy array (buffer) of char[2] byte strings
        - 'cut_idx': int, simulates the depth of the cut card
        - 'hrd': numpy array (buffer) of char bytes, in the input format dictating BJ action on player hard hand
        - 'sft': numpy array (buffer) of char bytes, in the input format dictating BJ action on player soft hand
        - 'splt': numpy array (buffer) of char bytes, in the input format dictating BJ action on player paired hand
        - 'cnt': numpy array (buffer) of double (Python float6), in the input format, with count values
    """

    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef double count = 0 # card count

    # declare arrays to hold idxs of cards for player and dealer
    cdef hand_t phand
    cdef hand_t dhand 

    # play the shoe
    while dealt < cut_idx:
    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
        
        # init both our hands to blanks

        break


    print(bool(shoe[0].face == b'J'))

    # type cast into python bytes, so works as dict key
    # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
    print(dealer_idx.get(<bytes>(shoe[0].face)))


